### Leveraging AI to help people become more productive

Problem Statement : Write a function that computes the optimal reward function (Equation 10) for the version of the flight planning problem shown in Figure 1A where the total number of flights is known (i.e., a finite horizon MDP with gamma=1). Your function should take the remaining number of flights and the current location as input arguments and return the incentives for the choices that are available in that location. Write tests for your code and demonstrate that it is working.



Approach : I have considered the six locations in the flight planning problem as numbered from 0 to 5 starting from Jonesville as 0, followed by Smithsville as 1 and so on in a clockwise fashion with Williamsville as 5. Further a state is defined by two variables: location and number of flights remaining. The policy is to perform the action that has maximum rewards to get optimal state value functions.  


In [ ]:
import numpy as np

In [ ]:
#declaring variables of MDP like states, actions etc.
STATES = 6
ACTIONS = 2
DISCOUNT = 1

#assumed total number of flights as 20
TOTAL_FLIGHTS = 20

#destinations from one state to another
DESTNS = [np.array([3,5]),
          np.array([0,4]),
          np.array([1,5]),
          np.array([1,2]),
          np.array([0,3]),
          np.array([2,4])]

#rewards of different actions from a state
REWARDS = [np.array([-70,-30]),
           np.array([140,30]),
           np.array([-30,30]),
           np.array([-70,-30]),
           np.array([30,-30]),
           np.array([-70,-30])]

#matrix to store optimal value functions of states
new_state_values = np.zeros((STATES, TOTAL_FLIGHTS + 1))

In [ ]:
#defining terminal state when the number of remaining flights is 0
def is_terminal(state):
    x, y = state
    return (y == 0)

In [ ]:
#this function returns the reward and next state based on current state and action chosen
def step(state, action):
    if is_terminal(state):
        return state, 0
    
    x, y = state
    '''the next state is the destination according to the action chosen from current state 
       and the flights remaining in the next location is one less than the current state '''
    next_state, reward = (DESTNS[x][action], y-1), REWARDS[x][action]
    
    return next_state, reward


In [ ]:
#this function is used to compute optimal state value functions
def compute_state_value():
    
    while True:
        #matrix to store state value functions from previous iteration
        state_values = new_state_values.copy()
        old_state_values = state_values.copy()

        for i in range(STATES):
            for j in range(TOTAL_FLIGHTS + 1):
                value = 0
                #taking next states and rewards based on both actions possible from current state
                (next_i1, next_j1), reward1 = step([i, j], 0)
                (next_i2, next_j2), reward2 = step([i, j], 1)
                
                #calculating discounted rewards of both actions and optimal policy is choosing action with maximum returns.
                rew1 = reward1 + DISCOUNT * state_values[next_i1, next_j1]
                rew2 = reward2 + DISCOUNT * state_values[next_i2, next_j2]
                if rew1 > rew2 :
                    value = rew1
                else :
                    value = rew2
                
                #updating the matrix that stores optimal state value functions
                new_state_values[i, j] = value
        
        #terminating condition is that the state values are now being updated by values less than 10^-4 which is insignificant
        max_delta_value = abs(old_state_values - new_state_values).max()
        if max_delta_value < 1e-4:
            break

    print(new_state_values)

In [ ]:
def incentives(rem_no_of_flights, location) :
    
    #calculating optimal rewards as stated in equation 9 in the research paper.
    if rem_no_of_flights > 0 :
        rew1 = REWARDS[location][0] + DISCOUNT * new_state_values[DESTNS[location][0], rem_no_of_flights-1] - new_state_values[location, rem_no_of_flights]
        rew2 = REWARDS[location][1] + DISCOUNT * new_state_values[DESTNS[location][1], rem_no_of_flights-1] - new_state_values[location, rem_no_of_flights]
    
    else : 
        rew1 = REWARDS[location][0] 
        rew2 = REWARDS[location][1] 
        
    print( rew1, rew2 )

In [ ]:
if __name__ == '__main__':
    compute_state_value()
    incentives(10,1)
    incentives(1,3)
    incentives(15,5)
    incentives(5,4)
    incentives(0,0)


[[  0. -30. -60.   0.  10. -20.   0.  10.  20.   0.  10.  20.  30.  10.
   20.  30.  40.  20.  30.  40.  50.]
 [  0. 140. 110.  80. 140. 150. 120. 140. 150. 160. 140. 150. 160. 170.
  150. 160. 170. 180. 160. 170. 180.]
 [  0.  30. 110.  80.  70. 110. 120.  90. 110. 120. 130. 110. 120. 130.
  140. 120. 130. 140. 150. 130. 140.]
 [  0. -30.  70.  80.  50.  70.  80.  90.  70.  80.  90. 100.  80.  90.
  100. 110.  90. 100. 110. 120. 100.]
 [  0.  30.   0.  40.  50.  40.  40.  50.  60.  50.  50.  60.  70.  60.
   60.  70.  80.  70.  70.  80.  90.]
 [  0. -30.   0.  40.  10.  20.  40.  50.  20.  40.  50.  60.  40.  50.
   60.  70.  50.  60.  70.  80.  60.]]
0.0 -60.0
-40.0 0.0
0.0 -40.0
0.0 -20.0
-70 -30
